# Snorkel Workshop: Extracting Spouse Relations from the News
## Part 1: Snorkel API Cheat Sheet

Complete API documentation is available at:

http://snorkel.readthedocs.io/en/latest/

However, we provide several detailed examples below that are useful when you are using Snorkel for the first time. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# Connect to the database backend and initalize a Snorkel session
from lib.init import *

/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## 1: `Candidate` and `Span` Objects
----

`Candidate` objects represent potential mentions found in text and are a core abstraction used in Snorkel. Candidates consist of 1 or more `Span` objects. 

For real applications, we have to define a candidate relation type consisting named `Span` objects. In this API, we'll use the pre-defined Spouse candidate, initialized below:

In [2]:
try:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
except:
    print>>sys.stderr,"Info: Candidate type already defined"

### Querying `Candidate` Sets
We assume that our Candidates have already been extracted and partitioned
into `train`, `dev`, and `test` sets.

See our preprocessing tutorial <a href="Workshop_Advanced_Preprocessing.ipynb">Workshop_Advanced_Preprocessing.ipynb</a> for more information on how candidates are generated. 

For now, we will just load our `train` set candidates.

In [3]:
cands = session.query(Candidate).filter(Candidate.split == 0).all()

### Accessing `Span` Arguments
The `Spouse` example above consists of 2 arguments: `person1` and `person2`. These are represented as `Span` objects. We can access these directy as member variables of a candidate as so:

In [4]:
c = cands[134]

In [5]:
print c.person1
print c.person2

print c[0], c[1]

Span("Derek Hough", sentence=68850, chars=[120,130], words=[22,23])
Span("Bindi", sentence=68850, chars=[37,41], words=[4,4])
Span("Derek Hough", sentence=68850, chars=[120,130], words=[22,23]) Span("Bindi", sentence=68850, chars=[37,41], words=[4,4])


### `Span` Member Functions
`Span` objects provide several useful functions for accessing text attributes.

In [6]:
print "Span 0 words               \t", c[0].get_attrib_tokens("words")
print "Span 0 lemmas              \t", c[0].get_attrib_tokens("words")
print "Span 0 part-of-speech tags \t", c[0].get_attrib_tokens("pos_tags")

Span 0 words               	[u'Derek', u'Hough']
Span 0 lemmas              	[u'Derek', u'Hough']
Span 0 part-of-speech tags 	[u'NNP', u'NNP']


## 2: `Context`  Objects: `Sentence` and `Document`
----

All `Candidate` objects live within a `Context`. We'll focus on `Sentence` objects to start. We can access any candidate's *parent* with the following method:

In [7]:
sent = c.get_parent()
sent

Sentence(Document 2f02e6e0-fda5-4995-85c8-a976d46a3ebf,4,Actress and wildlife conservationist Bindi, 17, hit the stage with her Season 21 partner, five-time mirrorball champion Derek Hough, for an exciting jive that wowed the crowd.)

### Text and Syntactic Attributes
Text undergoes standard information extraction preprocessing to extract important syntatic attributes like part-of-speech tags and depenency tree information. You can access all of these attribtes for each token using these `Sentence` member variables:

In [8]:
print sent.words
print sent.lemmas
print sent.pos_tags
print sent.ner_tags
print sent.dep_parents
print sent.dep_labels

[u'Actress', u'and', u'wildlife', u'conservationist', u'Bindi', u',', u'17', u',', u'hit', u'the', u'stage', u'with', u'her', u'Season', u'21', u'partner', u',', u'five', u'-', u'time', u'mirrorball', u'champion', u'Derek', u'Hough', u',', u'for', u'an', u'exciting', u'jive', u'that', u'wowed', u'the', u'crowd', u'.']
[u'actress', u'and', u'wildlife', u'conservationist', u'bindi', u',', u'17', u',', u'hit', u'the', u'stage', u'with', u'-PRON-', u'season', u'21', u'partner', u',', u'five', u'-', u'time', u'mirrorball', u'champion', u'derek', u'hough', u',', u'for', u'an', u'exciting', u'jive', u'that', u'wow', u'the', u'crowd', u'.']
[u'NN', u'CC', u'NN', u'NN', u'NNP', u',', u'CD', u',', u'VBD', u'DT', u'NN', u'IN', u'PRP$', u'NNP', u'CD', u'NN', u',', u'CD', u'HYPH', u'NN', u'NN', u'NN', u'NNP', u'NNP', u',', u'IN', u'DT', u'JJ', u'NN', u'WDT', u'VBD', u'DT', u'NN', u'.']
[u'O', u'O', u'O', u'O', u'PERSON', u'O', u'DATE', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'CARDINAL', u'O', u'

### Character-level Offsets
These lists give the starting offset (in characters) for each token in the `Sentence` object. 
`char_offsets` are relative to the start of the sentence while `abs_char_offsets` are releative to document. 

In [9]:
print sent.char_offsets
print sent.abs_char_offsets

[0, 8, 12, 21, 37, 42, 44, 46, 48, 52, 56, 62, 67, 71, 78, 81, 88, 90, 94, 95, 100, 111, 120, 126, 131, 133, 137, 140, 149, 154, 159, 165, 169, 174]
[606, 614, 618, 627, 643, 648, 650, 652, 654, 658, 662, 668, 673, 677, 684, 687, 694, 696, 700, 701, 706, 717, 726, 732, 737, 739, 743, 746, 755, 760, 765, 771, 775, 780]


### Visualizing Candidates
We can also visualize a candidate for purposes of debugging. 
This snippet shows the first 5 candidates in their parent sentences.

In [10]:
from lib.viz import *
display_candidate(c)

## 3: Helper functions

These are python helper functions that you can apply to candidates to return objects that are helpful during LF development.

You can (and should!) write your own helper functions to help write LFs.

In [11]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [12]:
print "Candidate LEFT tokens:   \t", list(get_left_tokens(c,window=2))
print "Candidate RIGHT tokens:  \t", list(get_right_tokens(c,window=2))
print "Candidate BETWEEN tokens:\t", get_text_between(c)

Candidate LEFT tokens:   	[u'mirrorball', u'champion']
Candidate RIGHT tokens:  	[u',', u'17']
Candidate BETWEEN tokens:	, 17, hit the stage with her Season 21 partner, five-time mirrorball champion 


### Advanced `Candidate` and `Sentence` Attributes

Snorkel creates a dependency tree representation for all `Sentence` objects. You can query
this using advanced lxml-based operators, but sometimes its also useful just to visualize a
sentence to identify important verbs, noun phrases, and other syntatic structure. 

In [13]:
from lib.tree_structs_ipynb import *
tree = sentence_to_xmltree(c.get_parent())

In [14]:
tree.render_tree()

<!--Provide the canvas id via python string formatting here--!>

## 4: `Cheat Sheet`
----

### Accessing `Candidate` tokens

Full helper function list at
http://snorkel.readthedocs.io/en/latest/etc.html#module-snorkel.lf_helpers

`snorkel.lf_helpers.get_left_tokens(...)
snorkel.lf_helpers.get_right_tokens(...)
snorkel.lf_helpers.get_between_tokens(...)
snorkel.lf_helpers.get_text_between(...)
snorkel.lf_helpers.get_tagged_text(...)`

### `Candidate` member functions

`c[0].get_attrib_tokens(...)
c[0].get_word_start(...)
c[0].get_word_end(...)`


### `Sentence` attributes

| Variable Name   | Description                          |
|-------------|------------------------------------------|
| `s.words`   | Text Tokens                              |
| `s.lemmas`  | Lemma, "a base word and its inflections" |
| `s.pos_tags` | Part-of-speech Tags                     |
| `s.ner_tags` | Named Entity Tags                     |
| `s.dep_parents` |  Dependency Tree Heads            |
| `s.dep_labels` |  Dependency Tree Tags            |  
| `s.char_offsets` |  Character Offsets          |
| `s.abs_char_offsets` |  Absolute (document) Character Offsets |


### Computing Labeling Function Metrics

`snorkel.lf_helpers.test_LF`